In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import xarray as xr
import zarr
from dask.distributed import Client
from joblib import Parallel, delayed
import pandas as pd

In [3]:
#client=Client(n_workers=8, threads_per_worker=1 )

In [4]:
#client

In [5]:
regions = {'Washington': ['Mt Hood', 'Olympics', 'Snoqualmie Pass', 'Stevens Pass',
           'WA Cascades East, Central', 'WA Cascades East, North', 'WA Cascades East, South',
           'WA Cascades West, Central', 'WA Cascades West, Mt Baker', 'WA Cascades West, South']}

In [6]:

date_values_pd = pd.date_range('2016-11-01', periods=181, freq="D")

In [7]:
date_values_pd[-10:]

DatetimeIndex(['2017-04-21', '2017-04-22', '2017-04-23', '2017-04-24',
               '2017-04-25', '2017-04-26', '2017-04-27', '2017-04-28',
               '2017-04-29', '2017-04-30'],
              dtype='datetime64[ns]', freq='D')

In [86]:
def compute_region(region_name, season='16-17', state='Washington'):
    first = True
    base_path = '/media/scottcha/E1/Data/OAPMLData/GFSTrainingData/GFSFiltered/' + season + '/' + state + '/Region_' + region_name 
    p = 181
    if season in ['15-16', '19-20']:
        p = 182 #leap years
    
    snow_start_date = '2015-11-01'
    if season == '16-17':
        snow_start_date = '2016-11-01'
    if season == '17-18':
        snow_start_date = '2017-11-01'
    if season == '18-19':
        snow_start_date = '2018-11-01'
    
    date_values_pd = pd.date_range(snow_start_date, periods=p, freq="D")
    try:
        with xr.open_zarr(base_path + '.zarr') as z:
            if z.time.values[-1] == date_values_pd[-1]:
                print(' already exists: ' + region_name + ' ' + season + ' ' + state)
                z.close()
                return
            else:
                #already exists but incomplete
                date_values_pd = [pd.Timestamp(v) for v in date_values_pd.values.astype('datetime64[ns]') if v not in z.time.values]
                print(' some exist but have to complete ' + str(len(date_values_pd)))
                first = False
    except ValueError as err:
        #ignore as it doesn't exist yet
        print('')
    
    for d in date_values_pd:
        
        path =  base_path + '_' + d.strftime('%Y%m%d') + '.nc'
        print('On ' + str(path.split('/')[-1]))
        
        try:
            ds = xr.open_dataset(path, chunks={'latitude':1, 'longitude':1})
        except OSError as err:
            print(' missing file: ' + path)
            continue
            
        ds = ds.to_array(name='vars').chunk({'time':1, 'latitude':1, 'longitude':1, 'variable':-1}).to_dataset()
        
        zarr_path = base_path + '.zarr'
        try:

            if first:
                ds.to_zarr(zarr_path, consolidated=True)
                first=False
            else:
                ds.to_zarr(zarr_path, consolidated=True, append_dim='time')
        except ValueError as err:
            print('Value Error on ' + zarr_path)
            return

def process_tuple(t):
    
    compute_region(t[0], t[1], t[2])

In [81]:
from datetime import datetime
season = '15-16'
state = 'Colorado'
region_name = 'Steamboat Zone'
base_path = '/media/scottcha/E1/Data/OAPMLData/GFSTrainingData/GFSFiltered/' + season + '/' + state + '/Region_' + region_name 
try:
    with xr.open_zarr(base_path + '.zarr') as z:
 
        date_values_pd = pd.date_range('2015-11-01', periods=182, freq="D")
        if z.time.values[-1] == date_values_pd[-1]:
            print(' already exists: ' + region_name + ' ' + season + ' ' + state)
            z.close()            
        else:
            #already exists but incomplete
            date_values_pd = [pd.Timestamp(v) for v in date_values_pd.values.astype('datetime64[ns]') if v not in z.time.values]
            print(' some exist but have to complete ' + str(len(date_values_pd)))
            
            first = False
        z.close()
except ValueError as err:
    print('Not found')


 some exist but have to complete 178


In [80]:
date_values_pd[0] == pd.Timestamp('2015-11-01 00:00:00')

True

In [61]:
z.time.values

array(['2015-11-01T00:00:00.000000000', '2015-11-01T00:00:00.000000000',
       '2015-11-05T00:00:00.000000000', '2015-11-06T00:00:00.000000000',
       '2015-11-08T00:00:00.000000000'], dtype='datetime64[ns]')

In [37]:
z.close()

In [49]:
regions = {'Utah': ['Abajos', 'Logan', 'Moab', 'Ogden', 'Provo', 
           'Salt Lake', 'Skyline', 'Uintas'],  
           'Colorado': ['Grand Mesa Zone', 'Sangre de Cristo Range', 'Steamboat Zone', 'Front Range Zone',
           'Vail Summit Zone', 'Sawatch Zone', 'Aspen Zone', 
           'North San Juan Mountains', 'South San Juan Mountains', 'Gunnison Zone'],
           'Washington': ['Mt Hood', 'Olympics', 'Snoqualmie Pass', 'Stevens Pass',
           'WA Cascades East, Central', 'WA Cascades East, North', 'WA Cascades East, South',
           'WA Cascades West, Central', 'WA Cascades West, Mt Baker', 'WA Cascades West, South'
           ]
           }
seasons = ['15-16', '16-17', '17-18', '18-19']

def make_list():
    to_process = []
    for s in seasons:
        for state in regions.keys():           
            for r in regions[state]:
                to_process.append((r,s,state))
    return to_process
    
                

In [50]:
l = make_list()

In [51]:
l

[('Abajos', '15-16', 'Utah'),
 ('Logan', '15-16', 'Utah'),
 ('Moab', '15-16', 'Utah'),
 ('Ogden', '15-16', 'Utah'),
 ('Provo', '15-16', 'Utah'),
 ('Salt Lake', '15-16', 'Utah'),
 ('Skyline', '15-16', 'Utah'),
 ('Uintas', '15-16', 'Utah'),
 ('Grand Mesa Zone', '15-16', 'Colorado'),
 ('Sangre de Cristo Range', '15-16', 'Colorado'),
 ('Steamboat Zone', '15-16', 'Colorado'),
 ('Front Range Zone', '15-16', 'Colorado'),
 ('Vail Summit Zone', '15-16', 'Colorado'),
 ('Sawatch Zone', '15-16', 'Colorado'),
 ('Aspen Zone', '15-16', 'Colorado'),
 ('North San Juan Mountains', '15-16', 'Colorado'),
 ('South San Juan Mountains', '15-16', 'Colorado'),
 ('Gunnison Zone', '15-16', 'Colorado'),
 ('Mt Hood', '15-16', 'Washington'),
 ('Olympics', '15-16', 'Washington'),
 ('Snoqualmie Pass', '15-16', 'Washington'),
 ('Stevens Pass', '15-16', 'Washington'),
 ('WA Cascades East, Central', '15-16', 'Washington'),
 ('WA Cascades East, North', '15-16', 'Washington'),
 ('WA Cascades East, South', '15-16', 'Washin

In [87]:
Parallel(n_jobs=15, backend="multiprocessing")(map(delayed(process_tuple), l))


On Region_Steamboat Zone_20151101.nc
 some exist but have to complete 152
On Region_Front Range Zone_20151201.nc
 already exists: Moab 15-16 Utah
 already exists: Provo 15-16 Utah
 already exists: Abajos 15-16 Utah
 already exists: Ogden 15-16 Utah
 already exists: Salt Lake 15-16 Utah
 already exists: Skyline 15-16 Utah
 already exists: Aspen Zone 15-16 Colorado
 already exists: Vail Summit Zone 15-16 Colorado
 already exists: Logan 15-16 Utah
 already exists: Sawatch Zone 15-16 Colorado
 already exists: Uintas 15-16 Utah
 already exists: Grand Mesa Zone 15-16 Colorado
 already exists: Sangre de Cristo Range 15-16 Colorado
 already exists: Gunnison Zone 15-16 Colorado
 already exists: Mt Hood 15-16 Washington
 already exists: Olympics 15-16 Washington
 already exists: Snoqualmie Pass 15-16 Washington
 already exists: Stevens Pass 15-16 Washington
 already exists: North San Juan Mountains 15-16 Colorado
 already exists: WA Cascades East, Central 15-16 Washington
 already exists: South

On Region_Gunnison Zone_20171109.nc
On Region_South San Juan Mountains_20170321.nc
On Region_Sawatch Zone_20171222.nc
On Region_Skyline_20180306.nc
On Region_Vail Summit Zone_20180110.nc
On Region_Aspen Zone_20171221.nc
On Region_Front Range Zone_20171117.nc
On Region_Gunnison Zone_20171110.nc
On Region_Steamboat Zone_20161215.nc
On Region_Sawatch Zone_20171223.nc
On Region_Aspen Zone_20171222.nc
On Region_Steamboat Zone_20171212.nc
On Region_Vail Summit Zone_20180111.nc
On Region_Front Range Zone_20170116.nc
On Region_Skyline_20180307.nc
On Region_South San Juan Mountains_20171106.nc
On Region_Sangre de Cristo Range_20180103.nc
On Region_North San Juan Mountains_20170312.nc
On Region_Aspen Zone_20171223.nc
On Region_Sangre de Cristo Range_20170312.nc
On Region_Gunnison Zone_20171111.nc
On Region_Vail Summit Zone_20180112.nc
On Region_Sawatch Zone_20171224.nc
On Region_South San Juan Mountains_20170322.nc
On Region_Skyline_20180308.nc
On Region_Aspen Zone_20171224.nc
On Region_Aspen Zo

On Region_Sawatch Zone_20180113.nc
On Region_Sangre de Cristo Range_20180111.nc
On Region_Vail Summit Zone_20180203.nc
On Region_South San Juan Mountains_20170331.nc
On Region_Aspen Zone_20180125.nc
On Region_Gunnison Zone_20171130.nc
On Region_Skyline_20180326.nc
On Region_North San Juan Mountains_20170320.nc
On Region_Sangre de Cristo Range_20170320.nc
On Region_Sawatch Zone_20180114.nc
On Region_Vail Summit Zone_20180204.nc
On Region_Aspen Zone_20180126.nc
On Region_WA Cascades East, North_20180202.nc
On Region_Aspen Zone_20180127.nc
On Region_Gunnison Zone_20171201.nc
On Region_Skyline_20180327.nc
On Region_South San Juan Mountains_20171117.nc
On Region_Vail Summit Zone_20180205.nc
On Region_Sawatch Zone_20180115.nc
On Region_Steamboat Zone_20161222.nc
On Region_Aspen Zone_20180128.nc
On Region_South San Juan Mountains_20170401.nc
On Region_Steamboat Zone_20171219.nc
On Region_Sangre de Cristo Range_20180112.nc
On Region_Gunnison Zone_20171202.nc
On Region_Vail Summit Zone_20180206

On Region_Steamboat Zone_20171225.nc
On Region_South San Juan Mountains_20171127.nc
On Region_Gunnison Zone_20171220.nc
On Region_Aspen Zone_20180301.nc
On Region_WA Cascades East, North_20180214.nc
On Region_Sawatch Zone_20180205.nc
 missing file: /media/scottcha/E1/Data/OAPMLData/GFSTrainingData/GFSFiltered/17-18/Colorado/Region_Sawatch Zone_20180205.nc
On Region_Sawatch Zone_20180206.nc
On Region_Vail Summit Zone_20180228.nc
On Region_Sangre de Cristo Range_20180121.nc
On Region_Skyline_20180416.nc
On Region_Aspen Zone_20180302.nc
On Region_Gunnison Zone_20171221.nc
On Region_Vail Summit Zone_20180301.nc
On Region_Aspen Zone_20180303.nc
On Region_Sawatch Zone_20180207.nc
On Region_Front Range Zone_20151214.nc
On Region_North San Juan Mountains_20170329.nc
On Region_South San Juan Mountains_20171128.nc
On Region_South San Juan Mountains_20170411.nc
On Region_Skyline_20180417.nc
On Region_WA Cascades East, North_20180215.nc
On Region_Aspen Zone_20180304.nc
On Region_Gunnison Zone_2017

On Region_WA Cascades East, North_20180226.nc
On Region_Gunnison Zone_20180109.nc
On Region_South San Juan Mountains_20171208.nc
On Region_Aspen Zone_20180404.nc
On Region_Sawatch Zone_20180228.nc
On Region_Front Range Zone_20151219.nc
On Region_Steamboat Zone_20170104.nc
On Region_Vail Summit Zone_20180324.nc
On Region_Steamboat Zone_20180101.nc
On Region_North San Juan Mountains_20170406.nc
On Region_Aspen Zone_20180405.nc
On Region_Gunnison Zone_20180110.nc
On Region_WA Cascades East, South_20180424.nc
On Region_Sangre de Cristo Range_20170406.nc
On Region_Aspen Zone_20180406.nc
On Region_Sawatch Zone_20180301.nc
On Region_Vail Summit Zone_20180325.nc
On Region_Front Range Zone_20171201.nc
On Region_WA Cascades East, North_20180227.nc
On Region_Sangre de Cristo Range_20180130.nc
On Region_South San Juan Mountains_20170421.nc
On Region_Gunnison Zone_20180111.nc
On Region_Aspen Zone_20180407.nc
On Region_South San Juan Mountains_20171209.nc
On Region_WA Cascades East, South_20180425.n

...

On Region_North San Juan Mountains_20190323.nc
On Region_WA Cascades West, Mt Baker_20190402.nc
On Region_South San Juan Mountains_20190407.nc
On Region_WA Cascades West, Central_20190318.nc
On Region_WA Cascades West, South_20190107.nc
On Region_WA Cascades West, Mt Baker_20190403.nc
On Region_WA Cascades East, South_20190408.nc
On Region_WA Cascades East, North_20190226.nc
On Region_WA Cascades East, Central_20190303.nc
On Region_WA Cascades West, Mt Baker_20190404.nc
On Region_WA Cascades West, Central_20190319.nc
On Region_Sangre de Cristo Range_20190426.nc
On Region_WA Cascades East, South_20190409.nc
On Region_North San Juan Mountains_20190324.nc
On Region_Steamboat Zone_20190317.nc
On Region_South San Juan Mountains_20190408.nc
On Region_WA Cascades West, South_20190108.nc
On Region_WA Cascades West, Mt Baker_20190405.nc
On Region_Front Range Zone_20190202.nc
On Region_WA Cascades East, North_20190227.nc
On Region_WA Cascades East, Central_20190304.nc
On Region_WA Cascades West,

On Region_WA Cascades West, Central_20190407.nc
On Region_WA Cascades East, South_20190430.nc
On Region_North San Juan Mountains_20190404.nc
On Region_South San Juan Mountains_20190421.nc
On Region_WA Cascades West, South_20190123.nc
On Region_Front Range Zone_20160418.nc
On Region_WA Cascades West, Central_20190408.nc
On Region_WA Cascades East, Central_20190320.nc
On Region_WA Cascades East, North_20190315.nc
On Region_Front Range Zone_20190208.nc
On Region_Steamboat Zone_20190326.nc
On Region_South San Juan Mountains_20190422.nc
On Region_North San Juan Mountains_20190405.nc
On Region_WA Cascades West, South_20190124.nc
On Region_WA Cascades West, Central_20190409.nc
On Region_WA Cascades East, Central_20190321.nc
On Region_WA Cascades East, North_20190316.nc
On Region_WA Cascades West, Central_20190410.nc
On Region_WA Cascades West, South_20190125.nc
On Region_South San Juan Mountains_20190423.nc
On Region_Front Range Zone_20180320.nc
On Region_North San Juan Mountains_20190406.nc


On Region_WA Cascades West, South_20190218.nc
On Region_Front Range Zone_20190218.nc
On Region_North San Juan Mountains_20190424.nc
On Region_Front Range Zone_20160429.nc
On Region_WA Cascades East, Central_20190417.nc
On Region_WA Cascades East, North_20190412.nc
On Region_WA Cascades West, South_20190219.nc
On Region_Steamboat Zone_20190410.nc
On Region_North San Juan Mountains_20190425.nc
On Region_WA Cascades East, Central_20190418.nc
On Region_WA Cascades East, North_20190413.nc
On Region_WA Cascades West, South_20190220.nc
On Region_Front Range Zone_20180330.nc
On Region_Front Range Zone_20160430.nc
On Region_Front Range Zone_20190219.nc
On Region_WA Cascades East, Central_20190419.nc
On Region_WA Cascades East, North_20190414.nc
On Region_North San Juan Mountains_20190426.nc
On Region_Steamboat Zone_20190411.nc
On Region_WA Cascades West, South_20190221.nc
On Region_WA Cascades East, Central_20190420.nc
On Region_WA Cascades East, North_20190415.nc
On Region_WA Cascades West, So

On Region_WA Cascades West, South_20190430.nc
On Region_Front Range Zone_20190319.nc
On Region_Front Range Zone_20180427.nc
On Region_Front Range Zone_20190320.nc
On Region_Front Range Zone_20180428.nc
On Region_Front Range Zone_20190321.nc
On Region_Front Range Zone_20180429.nc
On Region_Front Range Zone_20190322.nc
On Region_Front Range Zone_20180430.nc
On Region_Front Range Zone_20190323.nc


KeyError: '.zmetadata'

In [102]:
date_values_pd = pd.date_range('2019-03-23', periods=39, freq="D")
season = '18-19'
state = 'Colorado'
region_name = 'Front Range Zone'
base_path = '/media/scottcha/E1/Data/OAPMLData/GFSTrainingData/GFSFiltered/' + season + '/' + state + '/Region_' + region_name 

In [103]:
date_values_pd

DatetimeIndex(['2019-03-23', '2019-03-24', '2019-03-25', '2019-03-26',
               '2019-03-27', '2019-03-28', '2019-03-29', '2019-03-30',
               '2019-03-31', '2019-04-01', '2019-04-02', '2019-04-03',
               '2019-04-04', '2019-04-05', '2019-04-06', '2019-04-07',
               '2019-04-08', '2019-04-09', '2019-04-10', '2019-04-11',
               '2019-04-12', '2019-04-13', '2019-04-14', '2019-04-15',
               '2019-04-16', '2019-04-17', '2019-04-18', '2019-04-19',
               '2019-04-20', '2019-04-21', '2019-04-22', '2019-04-23',
               '2019-04-24', '2019-04-25', '2019-04-26', '2019-04-27',
               '2019-04-28', '2019-04-29', '2019-04-30'],
              dtype='datetime64[ns]', freq='D')

In [105]:
for d in date_values_pd:
        
    path =  base_path + '_' + d.strftime('%Y%m%d') + '.nc'
    print('On ' + str(path.split('/')[-1]))
        
    try:
        ds = xr.open_dataset(path, chunks={'latitude':1, 'longitude':1})
    except OSError as err:
        print(' missing file: ' + path)
        continue
            
    ds = ds.to_array(name='vars').chunk({'time':1, 'latitude':1, 'longitude':1, 'variable':-1}).to_dataset()
        
    zarr_path = base_path + '.zarr'
    try:

        if first:
            ds.to_zarr(zarr_path, consolidated=True)
            first=False
        else:
            ds.to_zarr(zarr_path, consolidated=True, append_dim='time')
    except ValueError as err:
        print('Value Error on ' + zarr_path)


On Region_Front Range Zone_20190323.nc
On Region_Front Range Zone_20190324.nc
On Region_Front Range Zone_20190325.nc
On Region_Front Range Zone_20190326.nc
On Region_Front Range Zone_20190327.nc
On Region_Front Range Zone_20190328.nc
On Region_Front Range Zone_20190329.nc
On Region_Front Range Zone_20190330.nc
On Region_Front Range Zone_20190331.nc
On Region_Front Range Zone_20190401.nc
On Region_Front Range Zone_20190402.nc
On Region_Front Range Zone_20190403.nc
On Region_Front Range Zone_20190404.nc
On Region_Front Range Zone_20190405.nc
On Region_Front Range Zone_20190406.nc
On Region_Front Range Zone_20190407.nc
On Region_Front Range Zone_20190408.nc
On Region_Front Range Zone_20190409.nc
On Region_Front Range Zone_20190410.nc
On Region_Front Range Zone_20190411.nc
On Region_Front Range Zone_20190412.nc
On Region_Front Range Zone_20190413.nc
On Region_Front Range Zone_20190414.nc
On Region_Front Range Zone_20190415.nc
On Region_Front Range Zone_20190416.nc
On Region_Front Range Zon

In [139]:
zds = xr.open_zarr('/media/scottcha/E1/Data/OAPMLData/GFSTrainingData/GFSFiltered/18-19/Colorado/Region_Front Range Zone.zarr', consolidated=True)

In [131]:
_, index = np.unique(zds['time'], return_index=True)

In [136]:
zds.isel(time=index).to_zarr('/media/scottcha/E1/Data/OAPMLData/GFSTrainingData/GFSFiltered/18-19/Colorado/Region_Front Range Zone2.zarr', mode='w', consolidated=True)

In [141]:
zds.chunks

Frozen(SortedKeysDict({'variable': (1131,), 'time': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), 'latitude': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), 'longitude': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)}))

In [108]:
t = zds.sel(latitude=48.88059, longitude=-120.084337, method='nearest')#, time=slice(10,20))

In [109]:
t

<xarray.Dataset>
Dimensions:    (time: 182, variable: 1131)
Coordinates:
    latitude   float64 40.99
    longitude  float64 -106.3
  * time       (time) datetime64[ns] 2018-11-01 2018-11-02 ... 2019-04-30
  * variable   (variable) <U52 'VIS_surface_min' ... 'LANDN_surface_avg'
Data variables:
    vars       (variable, time) float64 dask.array<chunksize=(1131, 1), meta=np.ndarray>

In [39]:
import numpy as np
date = np.datetime64('2016-01-12')
date2 = date - np.timedelta64(13, 'D')

In [40]:
date

numpy.datetime64('2016-01-12')

In [41]:
date2

numpy.datetime64('2015-12-30')

In [38]:
t.vars.sel(time=slice(date2, date))

<xarray.DataArray 'vars' (variable: 978, time: 14)>
dask.array<getitem, shape=(978, 14), dtype=float64, chunksize=(978, 1), chunktype=numpy.ndarray>
Coordinates:
    latitude   float64 48.88
    longitude  float64 -120.1
  * time       (time) datetime64[ns] 2015-12-30 2015-12-31 ... 2016-01-12
  * variable   (variable) <U52 'UGRD_planetaryboundarylayer_min' ... 'N5WAVH_500mb_avg'
Attributes:
    pyproj_srs:  +proj=longlat +datum=WGS84 +no_defs

In [30]:
t2 = zds.vars.sel(time=slice(date2, date))


In [31]:
t2

<xarray.DataArray 'vars' (variable: 978, time: 14, latitude: 5, longitude: 8)>
dask.array<getitem, shape=(978, 14, 5, 8), dtype=float64, chunksize=(978, 1, 1, 1), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 45.24 45.3 45.36 45.42 45.48
  * longitude  (longitude) float64 -121.9 -121.8 -121.8 ... -121.6 -121.5 -121.5
  * time       (time) datetime64[ns] 2015-12-19 2015-12-20 ... 2016-01-01
  * variable   (variable) <U52 'UGRD_planetaryboundarylayer_min' ... 'N5WAVH_500mb_avg'
Attributes:
    pyproj_srs:  +proj=longlat +datum=WGS84 +no_defs